In [101]:
import pandas as pd
import requests
import re
import numpy as np
    
def LoadTXTnSplit(url,regx='\|| \| |--|==',repl=[["\n\n","|"],["\n"," "]]):
    r = requests.get(url)
    T=r.text
    for ri in repl:
        T=T.replace(ri[0],ri[1])
    return pd.Series(re.split(regx,T))
    #T.replace("\n\n","|").replace("\n"," "))

K=LoadTXTnSplit("https://raw.githubusercontent.com/tobiasseidel/koalitionsvertrag2018/master/koalitionsvertrag2018.md")
K=K[K!=""]
S=LoadTXTnSplit("https://raw.githubusercontent.com/nicmer/analysis-zeitonline/master/electoralprograms/spd.txt")
U=LoadTXTnSplit("https://raw.githubusercontent.com/nicmer/analysis-zeitonline/master/electoralprograms/union.txt")
H=LoadTXTnSplit("http://pcai056.informatik.uni-leipzig.de/downloads/etc/legacy/Papers/top1000de.txt")
# K =?= ( S + U ) / 2 ?

## Analyse der Eingangstexte
### Zeichen pro Absatz

In [69]:
Vergleich=[['Koalitionsvertrag', K],['SPD', S],['Union', U]]
for N,X in Vergleich:
    print(N)
    txtLaenge=X.map(len)
    print("##Summe Zeichen "+str(sum(txtLaenge))+"\n##Zeichen/Absatz:")
    print(str(txtLaenge.describe().map(int))+"\n") # Texte haben alle etwa die gleiche Länge (pro Absatz)


Koalitionsvertrag
##Summe Zeichen 511575
##Zeichen/Absatz:
count    1743
mean      293
std       252
min         1
25%        99
50%       242
75%       404
max      1864
dtype: int64

SPD
##Summe Zeichen 289869
##Zeichen/Absatz:
count     687
mean      421
std       300
min         9
25%       198
50%       394
75%       600
max      2252
dtype: int64

Union
##Summe Zeichen 148150
##Zeichen/Absatz:
count     276
mean      536
std       804
min         0
25%        49
50%       301
75%       622
max      5959
dtype: int64



### Wortlänge

In [103]:
haeufig=H.str.split(" ",expand=True).stack()
for N,X in Vergleich:    
    print(N)
    Woerter = X.str.split(" ",expand=True).stack().map(lambda x: x.strip(" ()-.,\"#„—–"))
    WortLaenge=Woerter.map(len)
    print(str(WortLaenge.describe())+"\n")

Koalitionsvertrag
count    63878.000000
mean         6.907746
std          4.956456
min          0.000000
25%          3.000000
50%          6.000000
75%         10.000000
max         41.000000
dtype: float64

SPD
count    36762.000000
mean         6.772292
std          4.525831
min          0.000000
25%          3.000000
50%          6.000000
75%          9.000000
max         36.000000
dtype: float64

Union
count    19723.000000
mean         6.383055
std          4.254927
min          0.000000
25%          3.000000
50%          5.000000
75%          9.000000
max         30.000000
dtype: float64



### 20 häufigste Wörter (abgesehen von den 50 häufigsten Wörtern der dt. Sprache)

In [144]:
def WoerterVergleichbar(txt):
    return txt.strip(" ()-.,\"#„—–•").lower()

for N,X in Vergleich:    
    print("###"+N+"###")
    Woerter = X.str.split(" ",expand=True).stack().map(WoerterVergleichbar)
    nichtHaeufig=Woerter[np.logical_not(Woerter.isin(haeufig[0:50]))]
    W=nichtHaeufig.groupby(nichtHaeufig.tolist()).size().reset_index().rename(columns={0:'count', 'index':'txt'})
    print(W.sort_values(by="count",ascending=False).reset_index(drop=True)[0:20])
    print("Vokabular-Größe: "+str(W.shape[0]+50))

###Koalitionsvertrag###
             txt  count
0                  2127
1            wir   2104
2         wollen    905
3            zur    382
4          sowie    223
5        stärken    215
6           dass    196
7    deutschland    191
8          durch    187
9            uns    177
10  insbesondere    162
11  unterstützen    144
12          dazu    137
13         dabei    127
14       fördern    115
15        können    114
16      menschen    112
17        weiter    110
18      schaffen    110
19          mehr    108
Vokabular-Größe: 11132
###SPD###
             txt  count
0            wir   1119
1         wollen    377
2                   242
3           dass    187
4         können    134
5            uns    133
6           mehr    130
7       menschen    128
8           muss    122
9          durch    119
10        müssen    111
11           zur    105
12         dafür     98
13   deutschland     97
14          alle     94
15          ihre     87
16        sollen     86
17     

### Tf-idf Vergleich

In [281]:
def compareTfIdf(Txt1,Txt2):
    from sklearn.feature_extraction.text import TfidfVectorizer
    Tfidf = TfidfVectorizer().fit_transform(pd.concat([Txt1,Txt2]))
    pairwise_similarity = Tfidf * Tfidf.T
    return pairwise_similarity[0:len(Txt1),len(Txt1):] # top right sub-matrix
KSMat=compareTfIdf(K,S)
KS=pd.DataFrame(np.argmax(KSMat.todense(),axis=1))
KUMat=compareTfIdf(K,U)
KU=pd.DataFrame(np.argmax(KUMat.todense(),axis=1))

In [282]:
KS[0:10]

,0
0,0
1,239
2,16
3,0
4,0
5,600
6,10
7,45
8,7
9,9


In [283]:
K[10]

'Wir erleben neue politische Zeiten mit vielfältigen Herausforderungen für Deutschland — sowohl international als auch national. Deutschland ist weltweit ein anerkannter Partner, aber nur mit einem neuen Aufbruch für Europa wird Deutschland langfristig Frieden, Sicherheit und Wohlstand garantieren können. Die Europäische Union muss ihre Werte und ihr Wohlstandsversprechen bewahren und erneuern. Nur eine starke Europäische Union ist der Garant für eine Zukunft in Frieden, Sicherheit und Wohlstand.'

In [286]:
S[KS.iloc[10][0]]

'- Die besten Schulen. Mit unserem Schulmodernisierungsprogramm sorgen wir für gut ausgerüstete Klassenzimmer, digitale Bildung und modernste Ausstattung. Mit unserem Berufsschulpakt gehen wir die dringend notwendige Modernisierung der Berufsschulen an. - Kinderbetreuung: Wir sorgen für ein besseres Angebot an Krippen, Kitas, Kindertagespflege, Horten und Ganztagsschulen. Die Kita-Gebühren schaffen wir ab. - Investitionen in Fachkräfte: Wir machen Bildung und Ausbildung gebührenfrei – die Ausbildung ebenso wie das Erststudium bis zum Master und zur Meister- und Technikerprüfung. - Neue und bezahlbare Wohnungen. Wir werden den öffentlichen und sozialen Wohnungsbau ausbauen. Den Erwerb von Wohneigentum für Familien mit niedrigen und mittleren Einkommen werden wir durch ein Familienbaugeld erleichtern. - Moderne Verkehrswege. Wir wollen moderne Straßen, Schienen, Wasserstraßen und Luftverkehr, die eine digitale, schadstoffarme und sichere Mobilität für alle ermöglichen – egal ob in der St

In [287]:
U[KU.iloc[10][0]]

'- Unser Wohlstand und unsere Lebensqualität hängen wesentlich vom stetigen und nachhaltigen Wachstum unserer Wirtschaft ab. Sie muss international wettbewerbsfähig bleiben, ihre Fähigkeit zur Innovation ausbauen und ausreichend neue Arbeitsplätze schaffen. Auch und gerade im digitalen Zeitalter und im Zeitalter der Globalisierung. - Klimaschutz und Schutz der natürlichen Lebensgrundlagen sind existenziell wichtig. Weltweit wachsen Bevölkerung und Wohlstand. Dadurch steigen der Verbrauch an Rohstoffen und der Ausstoß von CO2 Wir müssen beweisen, dass intakte Umwelt, Wachstum und Wohlstand keine Gegensätze, sondern zwei Seiten derselben Medaille sind. Deshalb geht Europa beim Klimaschutz und Umweltschutz voran, deshalb stehen wir zum weltweiten Klima-Abkommen von Paris. - Sicherheit und Stabilität sind Voraussetzung für jede Art von gutem Leben und wirtschaftlichen Erfolg. CDU und CSU sind die Parteien der inneren Sicherheit. Nur ein handlungsfähiger Staat kann für Freiheit und Sicherhe

... hm .. funktioniert noch nicht so gut